In [ ]:
import paramiko
import subprocess

ip = "172.16.16.24"
user = "root"
pw = "asd123!@"
cmd = ""

def cc(cmd):
    cli = paramiko.SSHClient()
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    cli.connect(ip, username=user, password=pw, port=22)
    stdin, stdout, stderr = cli.exec_command(cmd, get_pty=True)  # ← 여기가 핵심
    out = stdout.read().decode()
    err = stderr.read().decode()
    print(out if out else err)
    cli.close()

def install_mariaDB():
    print("MariaDB를 설치합니다...")
  #  print(cc("apt update -y"))
    cc("apt install -y apache2 expect mariadb-server")
    cc("apt install -y mariadb-client-compat")
    cc("systemctl enable mariadb --now")
    print("설치완료~")
    
    
def set_init():
    print(cc("ls -l"))
    print("secure_installation 시작")
    MYSQL_ROOT_PASSWORD="asd123!@"
    cmd = f'''
cat <<EOF > /root/mysql_secure.exp
#!/usr/bin/expect -f
set timeout -1
set password [lindex $argv 0]

spawn mysql_secure_installation

expect "Enter current password for root (enter for none):"
send "\\r"
expect "Switch to unix_socket authentication"
send "Y\\r"
expect "Change the root password?"
send "y\\r"
expect "New password:"
send "{MYSQL_ROOT_PASSWORD}\\r"
expect "Re-enter new password:"
send "{MYSQL_ROOT_PASSWORD}\\r"
expect "Remove anonymous users?"
send "Y\\r"
expect "Disallow root login remotely?"
send "Y\\r"
expect "Remove test database and access to it?"
send "Y\\r"
expect "Reload privilege tables now?"
send "Y\\r"
expect eof
EOF

chmod +x /root/mysql_secure.exp
/usr/bin/expect /root/mysql_secure.exp "{MYSQL_ROOT_PASSWORD}"
'''
    print(cc(cmd))
    print("secure_installation 완료")

def go():
    install_mariaDB()
    set_init()

go()


MariaDB를 설치합니다...
